# Exploratory Data Analysis for Cab Investment Project, by Vinicius Brun, 2024

## What is the business problem?
    The business problem is to determine the best cab company for investment: pink company vs yellow company. 
    To do that, I research for references about techniques to compare two companies and found Ratio Analysis (Chen, 2023). These are a series of quantitaive indicators used by investor to compare companies. Not all of them can be used with our data, but some of them can:
    - Net Profit Margin = Net Profit ⁄ Total Revenue x 100
    - Operating Margin Ratio = Net Operating Income / Total Revenue 
    [Net Operating Income = Gross Income - Operating Expenses]

    Other ratios that could be calculated with more financial data are: Price-to-Earnings Ratio, Return on Assets Ratio and Return on Equity Ratio. 

    Another important analysis is about the taxi market. Recently ride share apps have become proeminent worldwide, but data indicates that the cab market remains strong (Statista, 2024). Research indicates that these two categories appeal to different customers.
    "Young people may be gravitating towards ride-sourcing because it presents a cost advantage over conventional taxis and is much more convenient in terms of service delivery. However, their use of ride-hailing services does not seem to be significant enough for it to be a threat to conventional taxis since few reported using their apps as their primary mode of transport or with high regularity (less than 2 weeks). Respondents tended to use ride-sourcing services for trips which were more social and were more likely to use them on trips during weekends, further demonstrating its limited use" (Dzisi et al., 2020).

    This indicates that, even though lyft apps have gained stagering proporitons, taxi companies are still relevant for investment. 
    
    In this exercise, we used data from 2016 to 2018, before the COIVD-19 outbreak, so we will not be considering the pandemic in the analysis. 
    
## References:
Chen, J. (2023, March 23). How does ratio analysis make it easier to compare different companies? Investopedia. https://www.investopedia.com/ask/answers/032315/how-does-ratio-analysis-make-it-easier-compare-different-companies.asp
Dzisi, E. K., Ackaah, W., Aprimah, B. A., & Adjei, E. (2020). Understanding demographics of ride-sourcing and the factors that underlie its use among young people. Scientific African, 7, e00288. https://doi.org/10.1016/j.sciaf.2020.e00288
Meng, Chuan & Kaiyrbayeva, Ainur. (2024). FACTORS INFLUENCING THE DECISION TO INVEST. Izdenister natigeler. 566-572. 10.37884/2-2024/55.
Statista. (2024). Taxi - United States | Statista Market Forecast.https://www.statista.com/outlook/mmo/shared-mobility/taxi/united-states#revenue


In [2]:
# Import files

import pandas as pd

cab_data = pd.read_csv("DataSets/Cab_Data.csv")
transaction_data = pd.read_csv("DataSets/Transaction_ID.csv")
customer_data = pd.read_csv("DataSets/Customer_ID.csv")
city_data = pd.read_csv("DataSets/City.csv")

# Field names and data types

### Cab_Data.csv
- Transaction_ID: int
- Date_of_Travel: str (or int if it's an encoded date)
- Company: str
- City: str
- KM Travelled: float
- Price Charged: float
- Cost of Trip: float

### City.csv
- City: str
- Population: int
- Users: int

### Customer_ID.csv
- Customer_ID: int
- Gender: str
- Age: int
- Income (USD/Month): float

### Transaction_ID.csv
- Transaction_ID: int
- Customer_ID: int
- Payment_Mode: str

# Deduplication approach (from Data Intake Report)

o	Identify exact duplicates based on the column Transaction_ID in the file Transaction_ID.csv (unique identifier).
o	Identify exact duplicates based on the column Customer_ID in the file Customer_ID.csv (unique identifier).
o	Identify exact duplicates based on the column Transaction_ID in the file Cab_Data.csv.

In [9]:
# Data Deduplication

# Transaction_ID.csv
# Drop exact duplicates based on Transaction_ID
transaction_data_deduped = transaction_data.drop_duplicates(subset="Transaction ID")
print("Duplicates removed from Transaction_ID.csv:", len(transaction_data) - len(transaction_data_deduped))

# 2. Deduplicate Customer_ID.csv
# Drop exact duplicates based on Customer_ID
customer_data_deduped = customer_data.drop_duplicates(subset="Customer ID")
print("Duplicates removed from Customer_ID.csv:", len(customer_data) - len(customer_data_deduped))

# 3. Deduplicate Cab_Data.csv
# Drop duplicates based on columns that define a unique trip
cab_data_deduped = cab_data.drop_duplicates(subset="Transaction ID")
print("Duplicates removed from Cab_Data.csv:", len(cab_data) - len(cab_data_deduped))

Duplicates removed from Transaction_ID.csv: 0
Duplicates removed from Customer_ID.csv: 0
Duplicates removed from Cab_Data.csv: 0


# Data validation approach (from Data Intake Report)

o	Join Cab_Data.csv with Transaction_ID.csv on Transaction_ID to ensure that each trip in Cab_Data.csv has a corresponding transaction in Transactoin_ID.csv. 
o	Join Transaction_ID.csv with Customer_ID.csv on Customer_ID to ensure that transaction in Transactoin_ID.csv has a corresponding customer in Customer_ID.csv


In [14]:
# Data Validation

# Validate that each trip in Cab_Data.csv has a corresponding transaction in Transaction_ID.csv

# Merge Cab_Data.csv with Transaction_ID.csv on Transaction_ID
cab_transaction_merged = cab_data_deduped.merge(transaction_data_deduped, on="Transaction ID", how="left", indicator=True)

# Check for any trips in Cab_Data.csv without a matching transaction
missing_transactions = cab_transaction_merged[cab_transaction_merged["_merge"] == "left_only"]
print("Trips in Cab_Data.csv without corresponding transaction in Transaction_ID.csv:", len(missing_transactions))

# Remove the _merge column
cab_transaction_merged = cab_transaction_merged.drop(columns="_merge")

# Display any missing transactions
if not missing_transactions.empty:
    print("Trips missing in Transaction_ID.csv:")
    print(missing_transactions)

# Step 2: Validate that each transaction in Transaction_ID.csv has a corresponding customer in Customer_ID.csv

# Merge Transaction_ID.csv with Customer_ID.csv on Customer_ID
transaction_customer_merged = transaction_data_deduped.merge(customer_data_deduped, on="Customer ID", how="left", indicator=True)

# Check for any transactions without a matching customer
missing_customers = transaction_customer_merged[transaction_customer_merged["_merge"] == "left_only"]
print("Transactions in Transaction_ID.csv without corresponding customer in Customer_ID.csv:", len(missing_customers))

# Remove the _merge column
transaction_customer_merged = transaction_customer_merged.drop(columns="_merge")

# Display any missing customers (optional)
if not missing_customers.empty:
    print("Transactions missing in Customer_ID.csv:")
    print(missing_customers)

Trips in Cab_Data.csv without corresponding transaction in Transaction_ID.csv: 0
Transactions in Transaction_ID.csv without corresponding customer in Customer_ID.csv: 0


# Relationships across the files and determine which should be joined or appended

## Cab_Data.csv and Transaction_ID.csv
- Relationship: One-to-One
- Transaction_ID is the primary key in both files. We can join these two files on Transaction_ID.

## Transaction_ID.csv and Customer_ID.csv
- Relationship: Many-to-One
- Customer_ID is the primary key in Customer_ID.csv and a foreign key in Transaction_ID.csv. We can join these files using Customer_ID.

## Cab_Data.csv and City.csv
- Relationship: Many-to-One
- City is the primary key in City.csv and a common field in Cab_Data.csv. We can join these files using City.

## There are no files to append.

# Create master data

Apply joins described above.

In [16]:
# Create Master Data

# Join cab_transaction_merged with Customer_ID.csv on Customer_ID (Many-to-One)
cab_transaction_customer_merged = cab_transaction_merged.merge(customer_data_deduped, on="Customer ID", how="inner")
print("Joined with Customer_ID: ", len(cab_transaction_customer_merged), "records")

# Join the resulting data with City.csv on City (Many-to-One)
master_data = cab_transaction_customer_merged.merge(city_data, on="City", how="inner")
print("Joined with City data: ", len(master_data), "records")

# Display the first few rows of the master dataset
print("Master Data Preview:")
print(master_data.head())

# Final master data structure and count
print("Total number of rows in Master Data:", len(master_data))

master_data.to_csv("DataSets/Master_Data.csv", index=False)

Joined with Customer_ID:  359392 records
Joined with City data:  359392 records
Master Data Preview:
   Transaction ID  Date of Travel   Company        City  KM Travelled  \
0        10000011           42377  Pink Cab  ATLANTA GA         30.45   
1        10000012           42375  Pink Cab  ATLANTA GA         28.62   
2        10000013           42371  Pink Cab  ATLANTA GA          9.04   
3        10000014           42376  Pink Cab  ATLANTA GA         33.17   
4        10000015           42372  Pink Cab  ATLANTA GA          8.73   

   Price Charged  Cost of Trip  Customer ID Payment_Mode Gender  Age  \
0         370.95       313.635        29290         Card   Male   28   
1         358.52       334.854        27703         Card   Male   27   
2         125.20        97.632        28712         Cash   Male   53   
3         377.40       351.602        28020         Cash   Male   23   
4         114.62        97.776        27182         Card   Male   33   

   Income (USD/Month) Popul

# Field/feature transformations

## Cab_Data.csv
 - Create a new column trnasforming Date_of_Travel from integer to string for improved readability. Keep the original Date_of_Travel for ease of calculations. Also, extract day of the week, month and year.
 - Calculare Cost per KM dividing Cost per Trip by KM Travelled
 - Calculate Profit per Trip and Profit Margin Ratio
 - Split City into City_Name and State

## City.csv
 - Calculare user density by dividing Users by Population

## Customer_ID.csv
 - Categorize users by age group
 - Categorize by income bracket: low, medium, high

## Joins
 - Count transactions per customer
 - Calculate customer lifetime value by summing Price Charged across all their trips
 - Monthly and Yearly Trip Count
 - Day of the week preference


In [17]:
# 1. Date Transformations
# Convert Date_of_Travel from integer to string and extract day of week, month, and year
master_data['Date of Travel'] = master_data['Date of Travel'].astype(str)
master_data['Date of Travel'] = pd.to_datetime(master_data['Date of Travel'], errors='coerce', format='%Y%m%d')

# Extract day of the week, month, and year
master_data['Travel Day'] = master_data['Date of Travel'].dt.day_name()
master_data['Travel Month'] = master_data['Date of Travel'].dt.month
master_data['Travel Year'] = master_data['Date of Travel'].dt.year

# 2. Cost per KM
# Calculate Cost per KM by dividing Cost of Trip by KM Travelled
master_data['Cost per KM'] = master_data['Cost of Trip'] / master_data['KM Travelled']

# 3. Profit per Trip and Profit Margin Ratio
# Profit per Trip is Price Charged minus Cost of Trip
master_data['Profit per Trip'] = master_data['Price Charged'] - master_data['Cost of Trip']

# Profit Margin Ratio is Profit per Trip divided by Price Charged
master_data['Profit Margin Ratio'] = master_data['Profit per Trip'] / master_data['Price Charged']

# 4. Split City into City_Name and State
# Assume City column format is "CityName State"
master_data[['City Name', 'State']] = master_data['City'].str.split(' ', n=1, expand=True)

# 5. User Density
# Remove commas and convert Population and Users to integers, then calculate User Density
master_data['Population'] = master_data['Population'].str.replace(",", "").astype(int)
master_data['Users'] = master_data['Users'].str.replace(",", "").astype(int)
master_data['User Density'] = master_data['Users'] / master_data['Population']

# 6. Age Group Categorization
# Define age groups
age_bins = [0, 18, 25, 35, 45, 60, 100]
age_labels = ['<18', '18-25', '26-35', '36-45', '46-60', '60+']
master_data['Age Group'] = pd.cut(master_data['Age'], bins=age_bins, labels=age_labels)

# 7. Income Bracket Categorization
# Define income brackets (example thresholds, adjust as needed)
income_bins = [0, 5000, 15000, 30000, float('inf')]
income_labels = ['Low', 'Medium', 'High', 'Very High']
master_data['Income Bracket'] = pd.cut(master_data['Income (USD/Month)'], bins=income_bins, labels=income_labels)

# 8. Count Transactions per Customer
# Calculate transactions per customer
transactions_per_customer = master_data.groupby('Customer ID').size()
master_data = master_data.merge(transactions_per_customer.rename('Transaction Count'), on='Customer ID')

# 9. Customer Lifetime Value
# Calculate CLV by summing Price Charged across all trips for each customer
clv = master_data.groupby('Customer ID')['Price Charged'].sum()
master_data = master_data.merge(clv.rename('Customer Lifetime Value'), on='Customer ID')

# 10. Monthly and Yearly Trip Count
# Group by month and year for trip count
monthly_trip_count = master_data.groupby(['Travel Year', 'Travel Month']).size().rename('Monthly Trip Count')
yearly_trip_count = master_data.groupby('Travel Year').size().rename('Yearly Trip Count')

# Merge Monthly and Yearly Trip Count back to master_data
master_data = master_data.merge(monthly_trip_count, on=['Travel Year', 'Travel Month'], how='left')
master_data = master_data.merge(yearly_trip_count, on='Travel Year', how='left')

# 11. Day of the Week Preference
# Group by day of the week to count trips per day and identify preferences
day_of_week_preference = master_data.groupby('Travel Day').size().rename('Day of the Week Preference')
master_data = master_data.merge(day_of_week_preference, on='Travel Day', how='left')

# Display transformed master_data
print("Transformed Master Data:")
print(master_data.head())

master_data.to_csv("DataSets/Master_Data_Transformed.csv", index=False)

Transformed Master Data:
   Transaction ID Date of Travel   Company        City  KM Travelled  \
0        10000011            NaT  Pink Cab  ATLANTA GA         30.45   
1        10000012            NaT  Pink Cab  ATLANTA GA         28.62   
2        10000013            NaT  Pink Cab  ATLANTA GA          9.04   
3        10000014            NaT  Pink Cab  ATLANTA GA         33.17   
4        10000015            NaT  Pink Cab  ATLANTA GA          8.73   

   Price Charged  Cost of Trip  Customer ID Payment_Mode Gender  ...  \
0         370.95       313.635        29290         Card   Male  ...   
1         358.52       334.854        27703         Card   Male  ...   
2         125.20        97.632        28712         Cash   Male  ...   
3         377.40       351.602        28020         Cash   Male  ...   
4         114.62        97.776        27182         Card   Male  ...   

   City Name  State  User Density  Age Group Income Bracket  \
0    ATLANTA     GA      0.030312      26-35  